In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [7]:
import pandas as pd

# Load the CSV file with robust handling
try:
    df = pd.read_csv(
        '/content/data.csv',  # Path to the file
        on_bad_lines='skip',  # Skip problematic rows
        escapechar="\\",      # Handle escape characters
        encoding='utf-8',     # Ensure UTF-8 encoding
        engine='python'       # Use Python engine for better error handling
    )
    print("CSV file loaded successfully!")
except Exception as e:
    print(f"Error loading CSV file: {e}")

# Display the first few rows to verify
df.head()


CSV file loaded successfully!


,topic,source,bias,url,title,date,authors,content,content_original,source_url,bias_text,ID
0,politics,Politico,0,https://www.politico.com/news/magazine/2019/11...,Can the Impeachment Hearings Actually Change A...,2019-11-13,NaN,"This happens for different reasons , but a key...","This happens for different reasons, but a key ...",www.politico.com,left,bpp2Cqpvyi2ER5Lr
1,china,Bloomberg,1,https://www.bloomberg.com/news/articles/2018-0...,Trump’s Tariff Barrage Pushes China Fight to P...,2018-07-11,"Brendan Scott, Enda Curran",LISTEN TO ARTICLE 5:37 SHARE THIS ARTICLE Shar...,LISTEN TO ARTICLE 5:37 SHARE THIS ARTICLE Shar...,www.bloomberg.com,center,PnVV1v7F0D8bhd5X
2,terrorism,Washington Times,2,http://www.washingtontimes.com/news/2014/sep/1...,Top general: Failed Foley rescue was ‘highest-...,2014-09-16,Stephen Dinan,The Pentagon ’ s top general said Tuesday that...,The Pentagon’s top general said Tuesday that t...,www.washingtontimes.com,right,pZlutCuZNm832qqK
3,politics,Salon,0,http://www.salon.com/2014/11/12/americas_post_...,America’s post-midterm inferno: Tea Party garb...,2014-11-12,Robert Hennelly,A full week after the alleged “ shellacking “ ...,A full week after the alleged “shellacking “ o...,www.salon.com,left,aMGkvFlpLyQrX6iu
4,politics,CNN (Web News),0,http://www.cnn.com/2014/01/08/politics/christi...,E-mails among Christie appointees suggest poli...,2014-01-08,Catherine E. Shoichet,Story highlights Gergen : `` It feeds into thi...,"Story highlights Gergen: ""It feeds into this n...",www.cnn.com,left,Ludq48gCzDiKtnmX


In [8]:
df = df.drop(columns=['source_url', 'url','content_original','authors'])

In [9]:
df

,topic,source,bias,title,date,content,bias_text,ID
0,politics,Politico,0,Can the Impeachment Hearings Actually Change A...,2019-11-13,"This happens for different reasons , but a key...",left,bpp2Cqpvyi2ER5Lr
1,china,Bloomberg,1,Trump’s Tariff Barrage Pushes China Fight to P...,2018-07-11,LISTEN TO ARTICLE 5:37 SHARE THIS ARTICLE Shar...,center,PnVV1v7F0D8bhd5X
2,terrorism,Washington Times,2,Top general: Failed Foley rescue was ‘highest-...,2014-09-16,The Pentagon ’ s top general said Tuesday that...,right,pZlutCuZNm832qqK
3,politics,Salon,0,America’s post-midterm inferno: Tea Party garb...,2014-11-12,A full week after the alleged “ shellacking “ ...,left,aMGkvFlpLyQrX6iu
4,politics,CNN (Web News),0,E-mails among Christie appointees suggest poli...,2014-01-08,Story highlights Gergen : `` It feeds into thi...,left,Ludq48gCzDiKtnmX
...,...,...,...,...,...,...,...,...
27973,politics,Fox Online News,2,Documents heighten scrutiny on Biden-Ukraine d...,NaN,"A batch of newly released documents , includin...",right,aF4XRLV95UJrubIv
27974,russia,NPR Online News,1,"Paul Manafort 'Brazenly' Broke The Law, Specia...",2019-02-23,"Paul Manafort 'Brazenly ' Broke The Law , Spec...",center,Fw8sQKJhdALBNpFT
27975,labor,CNBC,1,January adds a much stronger-than-expected 225...,2020-02-07,WASHINGTON — An unseasonably mild January help...,center,OjEZ9pSMXgM0CuDK
27976,economy_and_jobs,BBC News,1,US wage growth hits nine-year high,NaN,Wages in the US grew at their fastest pace for...,center,eoECafbg4LXnIoMm


In [10]:

# Combine 'title' and 'content' as input text
df['text'] = df['title'] + " " + df['content']

In [11]:
df

,topic,source,bias,title,date,content,bias_text,ID,text
0,politics,Politico,0,Can the Impeachment Hearings Actually Change A...,2019-11-13,"This happens for different reasons , but a key...",left,bpp2Cqpvyi2ER5Lr,Can the Impeachment Hearings Actually Change A...
1,china,Bloomberg,1,Trump’s Tariff Barrage Pushes China Fight to P...,2018-07-11,LISTEN TO ARTICLE 5:37 SHARE THIS ARTICLE Shar...,center,PnVV1v7F0D8bhd5X,Trump’s Tariff Barrage Pushes China Fight to P...
2,terrorism,Washington Times,2,Top general: Failed Foley rescue was ‘highest-...,2014-09-16,The Pentagon ’ s top general said Tuesday that...,right,pZlutCuZNm832qqK,Top general: Failed Foley rescue was ‘highest-...
3,politics,Salon,0,America’s post-midterm inferno: Tea Party garb...,2014-11-12,A full week after the alleged “ shellacking “ ...,left,aMGkvFlpLyQrX6iu,America’s post-midterm inferno: Tea Party garb...
4,politics,CNN (Web News),0,E-mails among Christie appointees suggest poli...,2014-01-08,Story highlights Gergen : `` It feeds into thi...,left,Ludq48gCzDiKtnmX,E-mails among Christie appointees suggest poli...
...,...,...,...,...,...,...,...,...,...
27973,politics,Fox Online News,2,Documents heighten scrutiny on Biden-Ukraine d...,NaN,"A batch of newly released documents , includin...",right,aF4XRLV95UJrubIv,Documents heighten scrutiny on Biden-Ukraine d...
27974,russia,NPR Online News,1,"Paul Manafort 'Brazenly' Broke The Law, Specia...",2019-02-23,"Paul Manafort 'Brazenly ' Broke The Law , Spec...",center,Fw8sQKJhdALBNpFT,"Paul Manafort 'Brazenly' Broke The Law, Specia..."
27975,labor,CNBC,1,January adds a much stronger-than-expected 225...,2020-02-07,WASHINGTON — An unseasonably mild January help...,center,OjEZ9pSMXgM0CuDK,January adds a much stronger-than-expected 225...
27976,economy_and_jobs,BBC News,1,US wage growth hits nine-year high,NaN,Wages in the US grew at their fastest pace for...,center,eoECafbg4LXnIoMm,US wage growth hits nine-year high Wages in th...


In [12]:
# Encode labels for bias and source
label_encoder_bias = LabelEncoder()
df['bias_label'] = label_encoder_bias.fit_transform(df['bias_text'])  # Encoding 'left', 'center', 'right'


In [13]:
label_encoder_source = LabelEncoder()
df['source_label'] = label_encoder_source.fit_transform(df['source'])  # Encoding publication names

In [14]:
df

,topic,source,bias,title,date,content,bias_text,ID,text,bias_label,source_label
0,politics,Politico,0,Can the Impeachment Hearings Actually Change A...,2019-11-13,"This happens for different reasons , but a key...",left,bpp2Cqpvyi2ER5Lr,Can the Impeachment Hearings Actually Change A...,1,271
1,china,Bloomberg,1,Trump’s Tariff Barrage Pushes China Fight to P...,2018-07-11,LISTEN TO ARTICLE 5:37 SHARE THIS ARTICLE Shar...,center,PnVV1v7F0D8bhd5X,Trump’s Tariff Barrage Pushes China Fight to P...,0,38
2,terrorism,Washington Times,2,Top general: Failed Foley rescue was ‘highest-...,2014-09-16,The Pentagon ’ s top general said Tuesday that...,right,pZlutCuZNm832qqK,Top general: Failed Foley rescue was ‘highest-...,3,410
3,politics,Salon,0,America’s post-midterm inferno: Tea Party garb...,2014-11-12,A full week after the alleged “ shellacking “ ...,left,aMGkvFlpLyQrX6iu,America’s post-midterm inferno: Tea Party garb...,1,303
4,politics,CNN (Web News),0,E-mails among Christie appointees suggest poli...,2014-01-08,Story highlights Gergen : `` It feeds into thi...,left,Ludq48gCzDiKtnmX,E-mails among Christie appointees suggest poli...,1,62
...,...,...,...,...,...,...,...,...,...,...,...
27973,politics,Fox Online News,2,Documents heighten scrutiny on Biden-Ukraine d...,NaN,"A batch of newly released documents , includin...",right,aF4XRLV95UJrubIv,Documents heighten scrutiny on Biden-Ukraine d...,3,138
27974,russia,NPR Online News,1,"Paul Manafort 'Brazenly' Broke The Law, Specia...",2019-02-23,"Paul Manafort 'Brazenly ' Broke The Law , Spec...",center,Fw8sQKJhdALBNpFT,"Paul Manafort 'Brazenly' Broke The Law, Specia...",0,233
27975,labor,CNBC,1,January adds a much stronger-than-expected 225...,2020-02-07,WASHINGTON — An unseasonably mild January help...,center,OjEZ9pSMXgM0CuDK,January adds a much stronger-than-expected 225...,0,60
27976,economy_and_jobs,BBC News,1,US wage growth hits nine-year high,NaN,Wages in the US grew at their fastest pace for...,center,eoECafbg4LXnIoMm,US wage growth hits nine-year high Wages in th...,0,28


In [15]:
# Split the dataset into training, validation, and test sets
train_texts, test_texts, train_bias_labels, test_bias_labels, train_source_labels, test_source_labels = train_test_split(
    df['text'], df['bias_label'], df['source_label'], test_size=0.2, random_state=42
)
val_texts, test_texts, val_bias_labels, test_bias_labels, val_source_labels, test_source_labels = train_test_split(
    test_texts, test_bias_labels, test_source_labels, test_size=0.5, random_state=42
)


In [16]:
# Load tokenizer and model for DistilBERT
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [17]:
# Tokenize the texts
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=512)

In [18]:
# Add 'bias_label' as the labels field in the datasets
train_dataset = Dataset.from_dict({
    'text': train_texts,
    'bias_label': train_bias_labels,
    'labels': train_bias_labels  # Add this field
})
val_dataset = Dataset.from_dict({
    'text': val_texts,
    'bias_label': val_bias_labels,
    'labels': val_bias_labels  # Add this field
})
test_dataset = Dataset.from_dict({
    'text': test_texts,
    'bias_label': test_bias_labels,
    'labels': test_bias_labels  # Add this field
})

# Tokenize and format the datasets
train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/22382 [00:00<?, ? examples/s]

Map:   0%|          | 0/2798 [00:00<?, ? examples/s]

Map:   0%|          | 0/2798 [00:00<?, ? examples/s]

In [ ]:
# Format datasets for PyTorch
#train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'bias_label', 'source_label'])
#val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'bias_label', 'source_label'])
#test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'bias_label', 'source_label'])


In [19]:
# Load models for DistilBERT
bias_model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder_bias.classes_))
source_model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_encoder_source.classes_))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    # Calculate accuracy
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

In [21]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save results and model
    eval_strategy="epoch",  # Evaluate model at the end of each epoch
    save_strategy="epoch",  # Save model at the end of each epoch
    logging_dir="./logs",  # Directory for logging
    learning_rate=2e-5,  # Learning rate
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=16,  # Batch size for evaluation
    num_train_epochs=3,  # Number of training epochs
    weight_decay=0.01,  # Weight decay (regularization)
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="f1",  # Metric to use for determining the best model
    save_total_limit=1  # Limit the number of saved checkpoints
)


## Bias Model

In [22]:
# Create Trainer instance for training the model
trainer_bias = Trainer(
    model=bias_model,  # The model to be trained
    args=training_args,  # Training arguments defined above
    train_dataset=train_dataset,  # Training dataset
    eval_dataset=val_dataset,  # Validation dataset
    tokenizer=tokenizer,  # Tokenizer for preprocessing
    compute_metrics=compute_metrics  # Function to calculate evaluation metrics
)
print("Training DistilBERT for Bias Classification...")
trainer_bias.train()

<ipython-input-22-93fc6f88aa1a>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_bias = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Training DistilBERT for Bias Classification...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.599200,0.569880,0.771623,0.771147,0.777734,0.771623
2,0.405800,0.525292,0.810936,0.810990,0.813197,0.810936
3,0.317200,0.612846,0.820944,0.820954,0.821606,0.820944


TrainOutput(global_step=8394, training_loss=0.4902701106331648, metrics={'train_runtime': 822.412, 'train_samples_per_second': 81.645, 'train_steps_per_second': 10.207, 'total_flos': 8894973197279232.0, 'train_loss': 0.4902701106331648, 'epoch': 3.0})

In [23]:
output_dir = "./saved_models"

# Save the model and tokenizer
trainer_bias.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Trainer model and tokenizer saved to {output_dir}")

Trainer model and tokenizer saved to ./saved_models


In [24]:
# Evaluate models
print("Evaluating Bias Classification Model...")
bias_metrics = trainer_bias.evaluate(test_dataset)

Evaluating Bias Classification Model...


In [25]:
print("Bias Classification Metrics:", bias_metrics)

Bias Classification Metrics: {'eval_loss': 0.6527548432350159, 'eval_accuracy': 0.8148677626876341, 'eval_f1': 0.8148371482244382, 'eval_precision': 0.8160834361690315, 'eval_recall': 0.8148677626876341, 'eval_runtime': 9.5669, 'eval_samples_per_second': 292.466, 'eval_steps_per_second': 18.292, 'epoch': 3.0}


In [26]:
# prompt: write code to highlight f1 score , accuracy precision recall from bias_metrics

# ... (Your existing code)

# Evaluate models
print("Evaluating Bias Classification Model...")
bias_metrics = trainer_bias.evaluate(test_dataset)
print("Bias Classification Metrics:", bias_metrics)

# Highlight key metrics
print("\nBias Classification Metrics (Highlighted):")
print(f"F1 Score: \033[92m{bias_metrics['eval_f1']:.4f}\033[0m")  # Green color for F1
print(f"Accuracy: \033[94m{bias_metrics['eval_accuracy']:.4f}\033[0m")  # Blue color for Accuracy
print(f"Precision: \033[93m{bias_metrics['eval_precision']:.4f}\033[0m")  # Yellow for Precision
print(f"Recall: \033[91m{bias_metrics['eval_recall']:.4f}\033[0m")  # Red for Recall

Evaluating Bias Classification Model...


Bias Classification Metrics: {'eval_loss': 0.6527548432350159, 'eval_accuracy': 0.8148677626876341, 'eval_f1': 0.8148371482244382, 'eval_precision': 0.8160834361690315, 'eval_recall': 0.8148677626876341, 'eval_runtime': 9.7572, 'eval_samples_per_second': 286.763, 'eval_steps_per_second': 17.936, 'epoch': 3.0}

Bias Classification Metrics (Highlighted):
F1 Score: 0.8148
Accuracy: 0.8149
Precision: 0.8161
Recall: 0.8149


## Source Model

In [27]:
# Train source model
trainer_source = Trainer(
    model=source_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Training DistilBERT for Source Classification...")
trainer_source.train()

<ipython-input-27-a6295e4d6e00>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_source = Trainer(


Training DistilBERT for Source Classification...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.611500,0.582124,0.762330,0.761445,0.772171,0.762330
2,0.430400,0.497622,0.803074,0.803133,0.804174,0.803074
3,0.339700,0.587830,0.815583,0.815576,0.815773,0.815583


TrainOutput(global_step=8394, training_loss=0.5643454354690431, metrics={'train_runtime': 778.4663, 'train_samples_per_second': 86.254, 'train_steps_per_second': 10.783, 'total_flos': 8960960598958080.0, 'train_loss': 0.5643454354690431, 'epoch': 3.0})

In [28]:
output_dir = "./saved_source_models"

# Save the model and tokenizer
trainer_source.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"Trainer model and tokenizer saved to {output_dir}")

Trainer model and tokenizer saved to ./saved_source_models


In [29]:
# Evaluate models
print("Evaluating Bias Classification Model...")
source_metrics = trainer_source.evaluate(test_dataset)
print("Bias Classification Metrics:", source_metrics)

# Highlight key metrics
print("\nBias Classification Metrics (Highlighted):")
print(f"F1 Score: \033[92m{source_metrics['eval_f1']:.4f}\033[0m")  # Green color for F1
print(f"Accuracy: \033[94m{source_metrics['eval_accuracy']:.4f}\033[0m")  # Blue color for Accuracy
print(f"Precision: \033[93m{source_metrics['eval_precision']:.4f}\033[0m")  # Yellow for Precision
print(f"Recall: \033[91m{source_metrics['eval_recall']:.4f}\033[0m")  # Red for Recall

Evaluating Bias Classification Model...


Bias Classification Metrics: {'eval_loss': 0.5608226656913757, 'eval_accuracy': 0.8302358827734095, 'eval_f1': 0.8302568532234459, 'eval_precision': 0.830407904909372, 'eval_recall': 0.8302358827734095, 'eval_runtime': 9.6053, 'eval_samples_per_second': 291.298, 'eval_steps_per_second': 18.219, 'epoch': 3.0}

Bias Classification Metrics (Highlighted):
F1 Score: 0.8303
Accuracy: 0.8302
Precision: 0.8304
Recall: 0.8302


##Examples

In [34]:
# prompt: i want to test the models on examples . write code for it

import torch

# Load the saved models and tokenizer
bias_model_path = "./saved_models"
source_model_path = "./saved_source_models"

bias_model = DistilBertForSequenceClassification.from_pretrained(bias_model_path)
source_model = DistilBertForSequenceClassification.from_pretrained(source_model_path)
tokenizer = DistilBertTokenizer.from_pretrained(bias_model_path)  # Use the same tokenizer

# Example texts
example_texts = ["4 ways Trumps mass deportation plans could hurt your finances President-elect Donald Trump has made tougher immigration enforcement a key campaign promise in each of his White House bids. If he follows through on his pledge for mass deportations and tighter immigration policies, it could create a financial burden for many Americans. Thats because immigrants including people who are in America illegally support the job market, keep inflation in check and add to the federal coffers. That contributes to the overall US economy If current immigration levels are sustained, the countrys real gross domestic product GDP could increase by 02 percentage points each year over the next decade, leaving it 2 higher in 2034, according to the Congressional Budget Offices outlook report for the next decade. By contrast, that projected growth would take a hit if Trump were able to enact his mass deportation plans. An estimated 11 million unauthorized immigrants reside in the United States, but Trumps focus has mostly been on deporting immigrants with criminal records. Goldman Sachs reported in June that it estimates about 12 million unauthorized immigrants, or 8 of that population, have criminal convictions. Heres the financial impact for Americans if Trump follows through on his mass deportation plan. Higher prices Deporting millions of undocumented workers would mean that businesses will need to replace those laborers. With historically low unemployment, finding people willing to work for low pay could be difficult, and companies may need to advertise higher wages to attract workers to replace deported laborers. Consumers would foot the bill if companies productivity slows or paychecks increase in the agriculture, construction and services sectors, which have a large number of undocumented workers. The University of New Hampshire Carsey School of Public Policy reported that the supply of goods would take a hit, similar to what happened during the pandemic. The report noted that inflation could peak at 05 percentage points higher under a mass deportation plan. Job growth will slow If huge populations of migrants are deported or at the very least population growth slows as a result of tighter immigration policies there will be less consumption of goods, which could hurt the labor market. Businesses would likely crack down on hiring due to a drop in revenue. Decreased consumer spending and demand could also lead to layoffs, as companies reassess their 2025 budgets, according to a Brookings Institute report. If there is negative net migration in 2025, the Brookings Institute estimates job growth would reduce by about 100000 a month. Lower pay Even as some people who fill the jobs left by deported laborers could get higher pay than the people they replace, in aggregate, deportations can lower paychecks for US-born workers. Between 2008 and 2015, wages for US-born workers decreased 06 after the deportation of 454000 unauthorized immigrant workers, according to a report by the University of New Hampshire Carsey School of Public Policy. What do you think Join 167 others in the comments View Comments The report said with fewer people consuming goods, job losses mounted across all skill levels, which offset gains by US-born low-skilled workers. Goldman Sachs said in a recent report that moderate fluctuations in immigration would likely have little impact on wage growth and inflation. The opposite would happen if there were dramatic policy changes. Social safety nets will take a hit If a large number of unauthorized workers are deported, it would cause funding problems for key federal programs. The American Immigration Council estimated in 2022 that unauthorized immigrants contributed 468 billion in federal taxes, with 226 billion going to Social Security and 57 billion going to Medicare. Unauthorized immigrants also paid 293 billion in state and local taxes. The deportation plan itself could be enormously expensive. Trump recently told NBC News that its not a question of a price tag for his plans for militarized deportations. The average cost of apprehending, detaining, processing and removing one undocumented immigrant in 2016 was 10900, according to figures released by US Immigration and Customs Enforcement ICE at the time. That year, ICE said the average cost of transporting one deportee to their home country was 1978. Since then, the costs have grown."]

# Tokenize and predict
for text in example_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Bias Prediction
    with torch.no_grad():
        bias_outputs = bias_model(**inputs)
        bias_predictions = torch.argmax(bias_outputs.logits, dim=-1).item()
    predicted_bias = label_encoder_bias.inverse_transform([bias_predictions])[0]

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_predictions = torch.argmax(source_outputs.logits, dim=-1).item()
    predicted_source = label_encoder_source.inverse_transform([source_predictions])[0]

    print(f"Text: {text}")
    print(f"Predicted Bias: {predicted_bias}")
    print(f"Predicted Source: {predicted_source}")
    print("-" * 20)

Text: 4 ways Trumps mass deportation plans could hurt your finances President-elect Donald Trump has made tougher immigration enforcement a key campaign promise in each of his White House bids. If he follows through on his pledge for mass deportations and tighter immigration policies, it could create a financial burden for many Americans. Thats because immigrants including people who are in America illegally support the job market, keep inflation in check and add to the federal coffers. That contributes to the overall US economy If current immigration levels are sustained, the countrys real gross domestic product GDP could increase by 02 percentage points each year over the next decade, leaving it 2 higher in 2034, according to the Congressional Budget Offices outlook report for the next decade. By contrast, that projected growth would take a hit if Trump were able to enact his mass deportation plans. An estimated 11 million unauthorized immigrants reside in the United States, but Trum

In [35]:
# Example texts
example_texts = ["Harris pins the blame on Trump for Supreme Court overturning Roe v Wade Vice President Kamala Harris on Monday pinned blame squarely on Donald Trump for the reversal of Roe v Wade saying the former president is proud that women are silently suffering without a guaranteed right to abortion The previous president expressed his intentions quite clearly And fast forward to just recently says hes proud of what he did Harris told CNNs Laura Coates during an exclusive interview in Wisconsin where the vice president was launching a national tour in support of reproductive rights She said the likely Republican nominee relishes his role in new restrictive abortion laws that have emerged in the aftermath of the high court ruling By inference he is proud that women have been deprived of fundamental freedoms to make decisions about their own body by inference proud that doctors are being penalized and criminalized for providing health care proud that women are silently suffering because they dont have access to the health care they need Harris added So lets understand that the stakes are so very high It was a preview of a central reelection argument in a rematch between Trump and President Joe Biden a contest that will be shadowed by the former presidents multiple courtroom trials including one related to Trumps role in events that led to the January 6 2021 riot at the Capitol The vice president rejected outright Trumps assertions that his legal issues amount to political persecution undertaken by the Biden administration What hes saying is not factual period she said And that would not be new for him would it And Harris said it was imperative Americans remain watchful for signs of democratic erosion in the leadup to this years contest"]

# Tokenize and predict
for text in example_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Bias Prediction
    with torch.no_grad():
        bias_outputs = bias_model(**inputs)
        bias_predictions = torch.argmax(bias_outputs.logits, dim=-1).item()
    predicted_bias = label_encoder_bias.inverse_transform([bias_predictions])[0]

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_predictions = torch.argmax(source_outputs.logits, dim=-1).item()
    predicted_source = label_encoder_source.inverse_transform([source_predictions])[0]

    print(f"Text: {text}")
    print(f"Predicted Bias: {predicted_bias}")
    print(f"Predicted Source: {predicted_source}")
    print("-" * 20)

Text: Harris pins the blame on Trump for Supreme Court overturning Roe v Wade Vice President Kamala Harris on Monday pinned blame squarely on Donald Trump for the reversal of Roe v Wade saying the former president is proud that women are silently suffering without a guaranteed right to abortion The previous president expressed his intentions quite clearly And fast forward to just recently says hes proud of what he did Harris told CNNs Laura Coates during an exclusive interview in Wisconsin where the vice president was launching a national tour in support of reproductive rights She said the likely Republican nominee relishes his role in new restrictive abortion laws that have emerged in the aftermath of the high court ruling By inference he is proud that women have been deprived of fundamental freedoms to make decisions about their own body by inference proud that doctors are being penalized and criminalized for providing health care proud that women are silently suffering because they 

In [38]:
example_texts=["US News Joe Rogan explains why liberal media hemorrhaging audiences Youre not accurate youre delusional And people are speaking with their subscriptions and theyre speaking with their purchasing of the Washington Post and their purchasing of the New York Times The conversation kicked off when Rogan 57 brought up Washington Post owner Jeff Bezos divisive October opinion piece The hard truth Americans dont trust the news media in which the billionaire Amazon founder declined to continue the newspapers legacy of endorsing a candidate for president The Washington Post planned to endorse Kamala Harris before Bezos stepped in claiming he move cost the newspaper thousands of subscribers but Rogan theorized it would have lost much more if it stuck to its progressive endorsement Essentially saying that you have to take divergent viewpoints you have to take a bunch of different perspectives we cant just be this leftwing echo chamber and its the reason why the business is faltering he noted The New York Times is suffering from the same leftleaning affliction Rogan argued pointing to a recent factcheck the newspaper published earlier this week on Robert F Kennedy Jrs claim that a popular breakfast cereal contains several artificial ingredients in the United States that are not used in other countriespolitical endorsements create a perception of bias The Gray Lady claimed Kennedy was wrong but admitted the ingredient lists for the USs version of Froot Loops contain a multitude of manmade chemicals that Canadas cereal does not The fact check is so dumb because the fact check says its not correct they have the same ingredients except for these harmful chemicals Rogan seethed adding that the ingredient list clearly included fng dangerous chemicals that are banned in Canada that were trying to get rid of in America So theyre literally saying he was wrong but he was right The popular podcaster theorized the Times went after RFK Jr so ruthlessly because of his close ties to Presidentelect Donald Trump and his nomination to serve as the next health secretary as well as his vocal aversion to vaccines Thats the New York fng Times he continued This is what the New York Times is doing so of course youre gonna hemorrhage subscribers of course Youre crazy youre saying something thats nuts and also What is your motivation Are you trying to eliminate because you lost so much credibility are you trying to kill it all Are you secretly working for the Chinese Like what are you doing Rogan lamented"]

# Tokenize and predict
for text in example_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Bias Prediction
    with torch.no_grad():
        bias_outputs = bias_model(**inputs)
        bias_predictions = torch.argmax(bias_outputs.logits, dim=-1).item()
    predicted_bias = label_encoder_bias.inverse_transform([bias_predictions])[0]

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_predictions = torch.argmax(source_outputs.logits, dim=-1).item()
    predicted_source = label_encoder_source.inverse_transform([source_predictions])[0]

    print(f"Text: {text}")
    print(f"Predicted Bias: {predicted_bias}")
    print(f"Predicted Source: {predicted_source}")
    print("-" * 20)

Text: US News Joe Rogan explains why liberal media hemorrhaging audiences Youre not accurate youre delusional And people are speaking with their subscriptions and theyre speaking with their purchasing of the Washington Post and their purchasing of the New York Times The conversation kicked off when Rogan 57 brought up Washington Post owner Jeff Bezos divisive October opinion piece The hard truth Americans dont trust the news media in which the billionaire Amazon founder declined to continue the newspapers legacy of endorsing a candidate for president The Washington Post planned to endorse Kamala Harris before Bezos stepped in claiming he move cost the newspaper thousands of subscribers but Rogan theorized it would have lost much more if it stuck to its progressive endorsement Essentially saying that you have to take divergent viewpoints you have to take a bunch of different perspectives we cant just be this leftwing echo chamber and its the reason why the business is faltering he noted

In [39]:
example_texts = ["Trump declaring national emergency at border would not lead to militarization of country expert says Fears over Presidentelect Trumps plan to declare a state of emergency at the border and make use of the military as part of a mass deportation plan have been overblown according to one expert The country is not going to be militarized Alfonso Aguilar a former chief of the US Office of Citizenship and the director of Hispanic engagement at the American Principles Project told Fox News Digital Neighborhoods with immigrants are not going to militarize The comments come after Trump confirmed Monday via Truth Social a report that he is prepared to declare a national emergency and will use military assets to reverse the Biden invasion through a mass deportation program The American people reelected President Trump by a resounding margin giving him a mandate to implement the promises he made on the campaign trail He will deliver Karoline Leavitt a spokesperson for the Trump transition team told Fox News Digital Such plans have been rumored for much of Trumps campaign raising fears among some critics that a future Trump administration would deploy the military onto American streets They are promising to use the military to do mass raids of American families at a scale that harkens back to some of the worst things our country has done Todd Schulte the president of FWDus an immigration advocacy organization told The Associated Press in October But Aguliar argued that the military is to play a support role providing logistics support to the US Immigrations and Customs Enforcement ICE and US Customs and Border Protection CBP There is a lot of fearmongering from the left and many in the media Aguliar said These are not going to be sweeps of neighborhoods these are going to be targeted arrests initially going after criminals I would anticipate that the military would participate in a supportive capacity he continued Youre detaining people youre going to need to move these people and provide shelter Aguilar pointed out that federal law limits how Trump can use the military domestically meaning ICE officers are likely to lead deportation efforts with a logistical hand from the military On the border the military would serve as a show of force while also helping be the eyes and ears for Border Patrol officers who are stretched thin across multiple sectors Thats historically how we use the military Aguliar said Force projection to dissuade people from coming in but their role is basically to help detect helping move migrants to buses all the logistical support Aguliar also argued that ICE is likely to make wider use of workplace raids where again the militarys logistical might could be harnessed to assist with detaining and transporting arrested migrants Theyre not going to set loose the military in neighborhoods trying to detect illegal immigrants Aguliar said Meanwhile many blue state leaders have announced plans to push back against Trumps deportations plans with Massachusetts Gov Maura Healey saying she is prepared to use every tool in the tool box to protect the states migrant residents and Illinois Gov JB Pritzker vowing to do everything that I can to protect our undocumented immigrants We cannot prohibit them federal law enforcement from coming into our state to you know conduct raids or do anything else like that Pritzker said last week Meanwhile I think it would be very difficult for them to just spread out across the country They dont have enough manpower within the Department of Homeland Security in order to carry that out"]

# Tokenize and predict
for text in example_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Bias Prediction
    with torch.no_grad():
        bias_outputs = bias_model(**inputs)
        bias_predictions = torch.argmax(bias_outputs.logits, dim=-1).item()
    predicted_bias = label_encoder_bias.inverse_transform([bias_predictions])[0]

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_predictions = torch.argmax(source_outputs.logits, dim=-1).item()
    predicted_source = label_encoder_source.inverse_transform([source_predictions])[0]

    print(f"Text: {text}")
    print(f"Predicted Bias: {predicted_bias}")
    print(f"Predicted Source: {predicted_source}")
    print("-" * 20)

Text: Trump declaring national emergency at border would not lead to militarization of country expert says Fears over Presidentelect Trumps plan to declare a state of emergency at the border and make use of the military as part of a mass deportation plan have been overblown according to one expert The country is not going to be militarized Alfonso Aguilar a former chief of the US Office of Citizenship and the director of Hispanic engagement at the American Principles Project told Fox News Digital Neighborhoods with immigrants are not going to militarize The comments come after Trump confirmed Monday via Truth Social a report that he is prepared to declare a national emergency and will use military assets to reverse the Biden invasion through a mass deportation program The American people reelected President Trump by a resounding margin giving him a mandate to implement the promises he made on the campaign trail He will deliver Karoline Leavitt a spokesperson for the Trump transition te

In [41]:
example_texts = ["The Irish Government Is Unbelievably Rich Its Largely Thanks to Uncle Sam DUBLINThe Irish government is rolling in clover like never before The country currently has so much money it pumps cash into not one but two sovereignwealth funds It is so flush that the budget watchdog doesnt warn about not having enough money but rather that the government is spending so much that it could overheat the economy In Dublin authorities are building what might become the worlds most expensive childrens hospital There are plans for a motorway to link Cork and Limerick new flood defenses in Shannon and floating wind farms off the south coast Outside the parliament sits a new bike shed that cost half a million dollars houses 36 bikes and doesnt keep out the rain The state is spending 10 million to get children off their phones at school including massbuying magnetic pouches to lock the devices away so they dont get distracted The good times are back says Pat Woods as he stretches his arms out over the red leather banquette of his pub the Dame Tavern in central Dublin Everything is flying Standing in a nearby street sucking on a vape a local hairdresser marvels at what is unfolding The spending is wild he says Helping fund this largess is the US tax system and an unexpected side effect of a global clampdown on corporate tax dodging The US government and the European Union spent the past decade changing laws and pressuring big multinationals not to book profits in offshore jurisdictions such as the Cayman Islands where they have no operations and pay no corporate tax So now many US companies are doing the next best thing parking their international profits in lowtax Ireland where they employ some people and pay some tax The Irish Fiscal Advisory Council estimates that just three big foreign companies made up 43 of corporation tax receipts in 2022 Around 15 of the Irish workforce is employed by just under 1000 US companies according to Irelands foreign direct investment agency Those businesses and the people employed by them may contribute as much as 60 of the governments entire tax take according to a recent estimate by Cormac Lucey a lecturer at Chartered Accountants Ireland"]

# Tokenize and predict
for text in example_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Bias Prediction
    with torch.no_grad():
        bias_outputs = bias_model(**inputs)
        bias_predictions = torch.argmax(bias_outputs.logits, dim=-1).item()
    predicted_bias = label_encoder_bias.inverse_transform([bias_predictions])[0]

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_predictions = torch.argmax(source_outputs.logits, dim=-1).item()
    predicted_source = label_encoder_source.inverse_transform([source_predictions])[0]

    print(f"Text: {text}")
    print(f"Predicted Bias: {predicted_bias}")
    print(f"Predicted Source: {predicted_source}")
    print("-" * 20)

Text: The Irish Government Is Unbelievably Rich Its Largely Thanks to Uncle Sam DUBLINThe Irish government is rolling in clover like never before The country currently has so much money it pumps cash into not one but two sovereignwealth funds It is so flush that the budget watchdog doesnt warn about not having enough money but rather that the government is spending so much that it could overheat the economy In Dublin authorities are building what might become the worlds most expensive childrens hospital There are plans for a motorway to link Cork and Limerick new flood defenses in Shannon and floating wind farms off the south coast Outside the parliament sits a new bike shed that cost half a million dollars houses 36 bikes and doesnt keep out the rain The state is spending 10 million to get children off their phones at school including massbuying magnetic pouches to lock the devices away so they dont get distracted The good times are back says Pat Woods as he stretches his arms out ove

## Conformal Prediction for source


In [44]:
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset

# If val_dataset is a Hugging Face Dataset, convert it to a standard Python list or NumPy array.
if isinstance(val_dataset, Dataset):
    val_dataset = val_dataset.to_pandas().values.tolist() # Convert to a list of lists or a numpy array

# Split validation dataset into calibration and evaluation sets
calibration_dataset, evaluation_dataset = train_test_split(val_dataset, test_size=0.5, random_state=42)


In [49]:
def compute_nonconformity_scores(trainer, dataset):
    model = trainer.model
    tokenizer = trainer.tokenizer
    model.eval()
    nonconformity_scores = []

    for batch in dataset:
        # Assuming 'text' is at index 0 in each sublist of dataset
        # Modify the index if 'text' is at a different position
        inputs = tokenizer(batch[0], return_tensors='pt', padding=True, truncation=True, max_length=512)
        inputs = {key: val.to(model.device) for key, val in inputs.items()}

        with torch.no_grad():
            logits = model(**inputs).logits
            probabilities = torch.softmax(logits, dim=-1)
            # Instead of len(batch[1]), use 1 to get the probability for the single label
            true_probs = probabilities[0, batch[1]]  # Get probability for the single label
            nonconformity_scores.append(1 - true_probs.cpu().item())  # Nonconformity: 1 - P(true_class)


    return np.array(nonconformity_scores)

In [50]:
# Compute nonconformity scores for the calibration dataset
calibration_nonconformity = compute_nonconformity_scores(trainer_source, calibration_dataset)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [51]:
# Step 2: Define Conformal Prediction Function
def conformal_prediction(trainer, inputs, calibration_nonconformity, confidence_level=0.9):
    model = trainer.model
    tokenizer = trainer.tokenizer
    model.eval()

    # Tokenize the input
    inputs = tokenizer(inputs, return_tensors='pt', padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(model.device) for key, val in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits
        probabilities = torch.softmax(logits, dim=-1).cpu().numpy()

    # Calculate the threshold from calibration nonconformity scores
    threshold = np.quantile(calibration_nonconformity, confidence_level)

    # Generate prediction sets
    prediction_sets = []
    for prob in probabilities:
        prediction_set = np.where(1 - prob <= threshold)[0]
        prediction_sets.append(prediction_set)

    return prediction_sets

In [56]:


# Assuming you have already defined tokenizer, bias_model, source_model, label_encoder_bias, and label_encoder_source

example_texts = ["Trump declaring national emergency at border would not lead to militarization of country expert says Fears over Presidentelect Trumps plan to declare a state of emergency at the border and make use of the military as part of a mass deportation plan have been overblown according to one expert The country is not going to be militarized Alfonso Aguilar a former chief of the US Office of Citizenship and the director of Hispanic engagement at the American Principles Project told Fox News Digital Neighborhoods with immigrants are not going to militarize The comments come after Trump confirmed Monday via Truth Social a report that he is prepared to declare a national emergency and will use military assets to reverse the Biden invasion through a mass deportation program The American people reelected President Trump by a resounding margin giving him a mandate to implement the promises he made on the campaign trail He will deliver Karoline Leavitt a spokesperson for the Trump transition team told Fox News Digital Such plans have been rumored for much of Trumps campaign raising fears among some critics that a future Trump administration would deploy the military onto American streets They are promising to use the military to do mass raids of American families at a scale that harkens back to some of the worst things our country has done Todd Schulte the president of FWDus an immigration advocacy organization told The Associated Press in October But Aguliar argued that the military is to play a support role providing logistics support to the US Immigrations and Customs Enforcement ICE and US Customs and Border Protection CBP There is a lot of fearmongering from the left and many in the media Aguliar said These are not going to be sweeps of neighborhoods these are going to be targeted arrests initially going after criminals I would anticipate that the military would participate in a supportive capacity he continued Youre detaining people youre going to need to move these people and provide shelter Aguilar pointed out that federal law limits how Trump can use the military domestically meaning ICE officers are likely to lead deportation efforts with a logistical hand from the military On the border the military would serve as a show of force while also helping be the eyes and ears for Border Patrol officers who are stretched thin across multiple sectors Thats historically how we use the military Aguliar said Force projection to dissuade people from coming in but their role is basically to help detect helping move migrants to buses all the logistical support Aguliar also argued that ICE is likely to make wider use of workplace raids where again the militarys logistical might could be harnessed to assist with detaining and transporting arrested migrants Theyre not going to set loose the military in neighborhoods trying to detect illegal immigrants Aguliar said Meanwhile many blue state leaders have announced plans to push back against Trumps deportations plans with Massachusetts Gov Maura Healey saying she is prepared to use every tool in the tool box to protect the states migrant residents and Illinois Gov JB Pritzker vowing to do everything that I can to protect our undocumented immigrants We cannot prohibit them federal law enforcement from coming into our state to you know conduct raids or do anything else like that Pritzker said last week Meanwhile I think it would be very difficult for them to just spread out across the country They dont have enough manpower within the Department of Homeland Security in order to carry that out"]  # Your list of example texts

# Tokenize and predict
for text in example_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # ... (Your existing bias prediction code)

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_predictions = torch.argmax(source_outputs.logits, dim=-1).item()
        # Get probabilities for all source labels
        source_probabilities = torch.softmax(source_outputs.logits, dim=-1)

    # Get the predicted source label
    predicted_source_label = label_encoder_source.inverse_transform([source_predictions])[0]

    # Get the top 3 most probable source labels and their probabilities
    top3_source_indices = torch.topk(source_probabilities, 3).indices[0].tolist()
    top3_source_probs = torch.topk(source_probabilities, 3).values[0].tolist()
    top3_source_labels = [label_encoder_source.inverse_transform([i])[0] for i in top3_source_indices]

    print(f"Text: {text}")
    #print(f"Predicted Bias: {predicted_bias}") # Assuming predicted_bias is defined in your existing code
    print(f"Predicted Source: {predicted_source_label}")
    print("Top 3 Source Predictions:")
    for label, prob in zip(top3_source_labels, top3_source_probs):
      print(f"- {label}: {prob:.4f}")
    print("-" * 20)

Text: Trump declaring national emergency at border would not lead to militarization of country expert says Fears over Presidentelect Trumps plan to declare a state of emergency at the border and make use of the military as part of a mass deportation plan have been overblown according to one expert The country is not going to be militarized Alfonso Aguilar a former chief of the US Office of Citizenship and the director of Hispanic engagement at the American Principles Project told Fox News Digital Neighborhoods with immigrants are not going to militarize The comments come after Trump confirmed Monday via Truth Social a report that he is prepared to declare a national emergency and will use military assets to reverse the Biden invasion through a mass deportation program The American people reelected President Trump by a resounding margin giving him a mandate to implement the promises he made on the campaign trail He will deliver Karoline Leavitt a spokesperson for the Trump transition te

### Example 2

In [57]:
example_texts = ["US News Joe Rogan explains why liberal media hemorrhaging audiences Youre not accurate youre delusional And people are speaking with their subscriptions and theyre speaking with their purchasing of the Washington Post and their purchasing of the New York Times The conversation kicked off when Rogan 57 brought up Washington Post owner Jeff Bezos divisive October opinion piece The hard truth Americans dont trust the news media in which the billionaire Amazon founder declined to continue the newspapers legacy of endorsing a candidate for president The Washington Post planned to endorse Kamala Harris before Bezos stepped in claiming he move cost the newspaper thousands of subscribers but Rogan theorized it would have lost much more if it stuck to its progressive endorsement Essentially saying that you have to take divergent viewpoints you have to take a bunch of different perspectives we cant just be this leftwing echo chamber and its the reason why the business is faltering he noted The New York Times is suffering from the same leftleaning affliction Rogan argued pointing to a recent factcheck the newspaper published earlier this week on Robert F Kennedy Jrs claim that a popular breakfast cereal contains several artificial ingredients in the United States that are not used in other countriespolitical endorsements create a perception of bias The Gray Lady claimed Kennedy was wrong but admitted the ingredient lists for the USs version of Froot Loops contain a multitude of manmade chemicals that Canadas cereal does not The fact check is so dumb because the fact check says its not correct they have the same ingredients except for these harmful chemicals Rogan seethed adding that the ingredient list clearly included fng dangerous chemicals that are banned in Canada that were trying to get rid of in America So theyre literally saying he was wrong but he was right The popular podcaster theorized the Times went after RFK Jr so ruthlessly because of his close ties to Presidentelect Donald Trump and his nomination to serve as the next health secretary as well as his vocal aversion to vaccines Thats the New York fng Times he continued This is what the New York Times is doing so of course youre gonna hemorrhage subscribers of course Youre crazy youre saying something thats nuts and also What is your motivation Are you trying to eliminate because you lost so much credibility are you trying to kill it all Are you secretly working for the Chinese Like what are you doing Rogan lamented"]

# Tokenize and predict
for text in example_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # ... (Your existing bias prediction code)

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_predictions = torch.argmax(source_outputs.logits, dim=-1).item()
        # Get probabilities for all source labels
        source_probabilities = torch.softmax(source_outputs.logits, dim=-1)

    # Get the predicted source label
    predicted_source_label = label_encoder_source.inverse_transform([source_predictions])[0]

    # Get the top 3 most probable source labels and their probabilities
    top3_source_indices = torch.topk(source_probabilities, 3).indices[0].tolist()
    top3_source_probs = torch.topk(source_probabilities, 3).values[0].tolist()
    top3_source_labels = [label_encoder_source.inverse_transform([i])[0] for i in top3_source_indices]

    print(f"Text: {text}")
    #print(f"Predicted Bias: {predicted_bias}") # Assuming predicted_bias is defined in your existing code
    print(f"Predicted Source: {predicted_source_label}")
    print("Top 3 Source Predictions:")
    for label, prob in zip(top3_source_labels, top3_source_probs):
      print(f"- {label}: {prob:.4f}")
    print("-" * 20)

Text: US News Joe Rogan explains why liberal media hemorrhaging audiences Youre not accurate youre delusional And people are speaking with their subscriptions and theyre speaking with their purchasing of the Washington Post and their purchasing of the New York Times The conversation kicked off when Rogan 57 brought up Washington Post owner Jeff Bezos divisive October opinion piece The hard truth Americans dont trust the news media in which the billionaire Amazon founder declined to continue the newspapers legacy of endorsing a candidate for president The Washington Post planned to endorse Kamala Harris before Bezos stepped in claiming he move cost the newspaper thousands of subscribers but Rogan theorized it would have lost much more if it stuck to its progressive endorsement Essentially saying that you have to take divergent viewpoints you have to take a bunch of different perspectives we cant just be this leftwing echo chamber and its the reason why the business is faltering he noted

In [58]:
example_texts = ["Harris pins the blame on Trump for Supreme Court overturning Roe v Wade Vice President Kamala Harris on Monday pinned blame squarely on Donald Trump for the reversal of Roe v Wade saying the former president is proud that women are silently suffering without a guaranteed right to abortion The previous president expressed his intentions quite clearly And fast forward to just recently says hes proud of what he did Harris told CNNs Laura Coates during an exclusive interview in Wisconsin where the vice president was launching a national tour in support of reproductive rights She said the likely Republican nominee relishes his role in new restrictive abortion laws that have emerged in the aftermath of the high court ruling By inference he is proud that women have been deprived of fundamental freedoms to make decisions about their own body by inference proud that doctors are being penalized and criminalized for providing health care proud that women are silently suffering because they dont have access to the health care they need Harris added So lets understand that the stakes are so very high It was a preview of a central reelection argument in a rematch between Trump and President Joe Biden a contest that will be shadowed by the former presidents multiple courtroom trials including one related to Trumps role in events that led to the January 6 2021 riot at the Capitol The vice president rejected outright Trumps assertions that his legal issues amount to political persecution undertaken by the Biden administration What hes saying is not factual period she said And that would not be new for him would it And Harris said it was imperative Americans remain watchful for signs of democratic erosion in the leadup to this years contest"]

# Tokenize and predict
for text in example_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # ... (Your existing bias prediction code)

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_predictions = torch.argmax(source_outputs.logits, dim=-1).item()
        # Get probabilities for all source labels
        source_probabilities = torch.softmax(source_outputs.logits, dim=-1)

    # Get the predicted source label
    predicted_source_label = label_encoder_source.inverse_transform([source_predictions])[0]

    # Get the top 3 most probable source labels and their probabilities
    top3_source_indices = torch.topk(source_probabilities, 3).indices[0].tolist()
    top3_source_probs = torch.topk(source_probabilities, 3).values[0].tolist()
    top3_source_labels = [label_encoder_source.inverse_transform([i])[0] for i in top3_source_indices]

    print(f"Text: {text}")
    #print(f"Predicted Bias: {predicted_bias}") # Assuming predicted_bias is defined in your existing code
    print(f"Predicted Source: {predicted_source_label}")
    print("Top 3 Source Predictions:")
    for label, prob in zip(top3_source_labels, top3_source_probs):
      print(f"- {label}: {prob:.4f}")
    print("-" * 20)

Text: Harris pins the blame on Trump for Supreme Court overturning Roe v Wade Vice President Kamala Harris on Monday pinned blame squarely on Donald Trump for the reversal of Roe v Wade saying the former president is proud that women are silently suffering without a guaranteed right to abortion The previous president expressed his intentions quite clearly And fast forward to just recently says hes proud of what he did Harris told CNNs Laura Coates during an exclusive interview in Wisconsin where the vice president was launching a national tour in support of reproductive rights She said the likely Republican nominee relishes his role in new restrictive abortion laws that have emerged in the aftermath of the high court ruling By inference he is proud that women have been deprived of fundamental freedoms to make decisions about their own body by inference proud that doctors are being penalized and criminalized for providing health care proud that women are silently suffering because they 

In [59]:
example_texts=[ "4 ways Trumps mass deportation plans could hurt your finances President-elect Donald Trump has made tougher immigration enforcement a key campaign promise in each of his White House bids. If he follows through on his pledge for mass deportations and tighter immigration policies, it could create a financial burden for many Americans. Thats because immigrants including people who are in America illegally support the job market, keep inflation in check and add to the federal coffers. That contributes to the overall US economy If current immigration levels are sustained, the countrys real gross domestic product GDP could increase by 02 percentage points each year over the next decade, leaving it 2 higher in 2034, according to the Congressional Budget Offices outlook report for the next decade. By contrast, that projected growth would take a hit if Trump were able to enact his mass deportation plans. An estimated 11 million unauthorized immigrants reside in the United States, but Trumps focus has mostly been on deporting immigrants with criminal records. Goldman Sachs reported in June that it estimates about 12 million unauthorized immigrants, or 8 of that population, have criminal convictions. Heres the financial impact for Americans if Trump follows through on his mass deportation plan. Higher prices Deporting millions of undocumented workers would mean that businesses will need to replace those laborers. With historically low unemployment, finding people willing to work for low pay could be difficult, and companies may need to advertise higher wages to attract workers to replace deported laborers. Consumers would foot the bill if companies productivity slows or paychecks increase in the agriculture, construction and services sectors, which have a large number of undocumented workers. The University of New Hampshire Carsey School of Public Policy reported that the supply of goods would take a hit, similar to what happened during the pandemic. The report noted that inflation could peak at 05 percentage points higher under a mass deportation plan. Job growth will slow If huge populations of migrants are deported or at the very least population growth slows as a result of tighter immigration policies there will be less consumption of goods, which could hurt the labor market. Businesses would likely crack down on hiring due to a drop in revenue. Decreased consumer spending and demand could also lead to layoffs, as companies reassess their 2025 budgets, according to a Brookings Institute report. If there is negative net migration in 2025, the Brookings Institute estimates job growth would reduce by about 100000 a month. Lower pay Even as some people who fill the jobs left by deported laborers could get higher pay than the people they replace, in aggregate, deportations can lower paychecks for US-born workers. Between 2008 and 2015, wages for US-born workers decreased 06 after the deportation of 454000 unauthorized immigrant workers, according to a report by the University of New Hampshire Carsey School of Public Policy. What do you think Join 167 others in the comments View Comments The report said with fewer people consuming goods, job losses mounted across all skill levels, which offset gains by US-born low-skilled workers. Goldman Sachs said in a recent report that moderate fluctuations in immigration would likely have little impact on wage growth and inflation. The opposite would happen if there were dramatic policy changes. Social safety nets will take a hit If a large number of unauthorized workers are deported, it would cause funding problems for key federal programs. The American Immigration Council estimated in 2022 that unauthorized immigrants contributed 468 billion in federal taxes, with 226 billion going to Social Security and 57 billion going to Medicare. Unauthorized immigrants also paid 293 billion in state and local taxes. The deportation plan itself could be enormously expensive. Trump recently told NBC News that its not a question of a price tag for his plans for militarized deportations. The average cost of apprehending, detaining, processing and removing one undocumented immigrant in 2016 was 10900, according to figures released by US Immigration and Customs Enforcement ICE at the time. That year, ICE said the average cost of transporting one deportee to their home country was 1978. Since then, the costs have grown."]
# Tokenize and predict
for text in example_texts:
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # ... (Your existing bias prediction code)

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_predictions = torch.argmax(source_outputs.logits, dim=-1).item()
        # Get probabilities for all source labels
        source_probabilities = torch.softmax(source_outputs.logits, dim=-1)

    # Get the predicted source label
    predicted_source_label = label_encoder_source.inverse_transform([source_predictions])[0]

    # Get the top 3 most probable source labels and their probabilities
    top3_source_indices = torch.topk(source_probabilities, 3).indices[0].tolist()
    top3_source_probs = torch.topk(source_probabilities, 3).values[0].tolist()
    top3_source_labels = [label_encoder_source.inverse_transform([i])[0] for i in top3_source_indices]

    print(f"Text: {text}")
    #print(f"Predicted Bias: {predicted_bias}") # Assuming predicted_bias is defined in your existing code
    print(f"Predicted Source: {predicted_source_label}")
    print("Top 3 Source Predictions:")
    for label, prob in zip(top3_source_labels, top3_source_probs):
      print(f"- {label}: {prob:.4f}")
    print("-" * 20)

Text: 4 ways Trumps mass deportation plans could hurt your finances President-elect Donald Trump has made tougher immigration enforcement a key campaign promise in each of his White House bids. If he follows through on his pledge for mass deportations and tighter immigration policies, it could create a financial burden for many Americans. Thats because immigrants including people who are in America illegally support the job market, keep inflation in check and add to the federal coffers. That contributes to the overall US economy If current immigration levels are sustained, the countrys real gross domestic product GDP could increase by 02 percentage points each year over the next decade, leaving it 2 higher in 2034, according to the Congressional Budget Offices outlook report for the next decade. By contrast, that projected growth would take a hit if Trump were able to enact his mass deportation plans. An estimated 11 million unauthorized immigrants reside in the United States, but Trum

In [61]:
example_texts=[ "4 ways Trumps mass deportation plans could hurt your finances President-elect Donald Trump has made tougher immigration enforcement a key campaign promise in each of his White House bids. If he follows through on his pledge for mass deportations and tighter immigration policies, it could create a financial burden for many Americans. Thats because immigrants including people who are in America illegally support the job market, keep inflation in check and add to the federal coffers. That contributes to the overall US economy If current immigration levels are sustained, the countrys real gross domestic product GDP could increase by 02 percentage points each year over the next decade, leaving it 2 higher in 2034, according to the Congressional Budget Offices outlook report for the next decade. By contrast, that projected growth would take a hit if Trump were able to enact his mass deportation plans. An estimated 11 million unauthorized immigrants reside in the United States, but Trumps focus has mostly been on deporting immigrants with criminal records. Goldman Sachs reported in June that it estimates about 12 million unauthorized immigrants, or 8 of that population, have criminal convictions. Heres the financial impact for Americans if Trump follows through on his mass deportation plan. Higher prices Deporting millions of undocumented workers would mean that businesses will need to replace those laborers. With historically low unemployment, finding people willing to work for low pay could be difficult, and companies may need to advertise higher wages to attract workers to replace deported laborers. Consumers would foot the bill if companies productivity slows or paychecks increase in the agriculture, construction and services sectors, which have a large number of undocumented workers. The University of New Hampshire Carsey School of Public Policy reported that the supply of goods would take a hit, similar to what happened during the pandemic. The report noted that inflation could peak at 05 percentage points higher under a mass deportation plan. Job growth will slow If huge populations of migrants are deported or at the very least population growth slows as a result of tighter immigration policies there will be less consumption of goods, which could hurt the labor market. Businesses would likely crack down on hiring due to a drop in revenue. Decreased consumer spending and demand could also lead to layoffs, as companies reassess their 2025 budgets, according to a Brookings Institute report. If there is negative net migration in 2025, the Brookings Institute estimates job growth would reduce by about 100000 a month. Lower pay Even as some people who fill the jobs left by deported laborers could get higher pay than the people they replace, in aggregate, deportations can lower paychecks for US-born workers. Between 2008 and 2015, wages for US-born workers decreased 06 after the deportation of 454000 unauthorized immigrant workers, according to a report by the University of New Hampshire Carsey School of Public Policy. What do you think Join 167 others in the comments View Comments The report said with fewer people consuming goods, job losses mounted across all skill levels, which offset gains by US-born low-skilled workers. Goldman Sachs said in a recent report that moderate fluctuations in immigration would likely have little impact on wage growth and inflation. The opposite would happen if there were dramatic policy changes. Social safety nets will take a hit If a large number of unauthorized workers are deported, it would cause funding problems for key federal programs. The American Immigration Council estimated in 2022 that unauthorized immigrants contributed 468 billion in federal taxes, with 226 billion going to Social Security and 57 billion going to Medicare. Unauthorized immigrants also paid 293 billion in state and local taxes. The deportation plan itself could be enormously expensive. Trump recently told NBC News that its not a question of a price tag for his plans for militarized deportations. The average cost of apprehending, detaining, processing and removing one undocumented immigrant in 2016 was 10900, according to figures released by US Immigration and Customs Enforcement ICE at the time. That year, ICE said the average cost of transporting one deportee to their home country was 1978. Since then, the costs have grown."]
# Tokenize and predict for each example
for text in example_texts:
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(source_model.device) for key, val in inputs.items()}  # Send inputs to model's device

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_probabilities = torch.softmax(source_outputs.logits, dim=-1)  # Compute probabilities for each class

    # Get the most probable source label
    source_predictions = torch.argmax(source_probabilities, dim=-1).item()  # Get the index of the highest probability
    predicted_source_label = label_encoder_source.inverse_transform([source_predictions])[0]  # Decode the label

    # Get the top 3 most probable source labels and their probabilities
    top3_source_indices = torch.topk(source_probabilities, 3).indices[0].tolist()  # Indices of top 3 classes
    top3_source_probs = torch.topk(source_probabilities, 3).values[0].tolist()  # Probabilities of top 3 classes
    top3_source_labels = label_encoder_source.inverse_transform(top3_source_indices)  # Decode top 3 labels

    # Print the results
    print(f"Text: {text[:100]}...")  # Print first 100 characters of the text for brevity
    print(f"Predicted Source: {predicted_source_label}")
    print("Top 3 Source Predictions:")
    for label, prob in zip(top3_source_labels, top3_source_probs):
        print(f"- {label}: {prob:.4f}")
    print("-" * 20)


Text: 4 ways Trumps mass deportation plans could hurt your finances President-elect Donald Trump has made ...
Predicted Source: AARP
Top 3 Source Predictions:
- AARP: 0.6717
- ABC News: 0.2414
- ACLU: 0.0868
--------------------


In [62]:
example_texts = ["Harris pins the blame on Trump for Supreme Court overturning Roe v Wade Vice President Kamala Harris on Monday pinned blame squarely on Donald Trump for the reversal of Roe v Wade saying the former president is proud that women are silently suffering without a guaranteed right to abortion The previous president expressed his intentions quite clearly And fast forward to just recently says hes proud of what he did Harris told CNNs Laura Coates during an exclusive interview in Wisconsin where the vice president was launching a national tour in support of reproductive rights She said the likely Republican nominee relishes his role in new restrictive abortion laws that have emerged in the aftermath of the high court ruling By inference he is proud that women have been deprived of fundamental freedoms to make decisions about their own body by inference proud that doctors are being penalized and criminalized for providing health care proud that women are silently suffering because they dont have access to the health care they need Harris added So lets understand that the stakes are so very high It was a preview of a central reelection argument in a rematch between Trump and President Joe Biden a contest that will be shadowed by the former presidents multiple courtroom trials including one related to Trumps role in events that led to the January 6 2021 riot at the Capitol The vice president rejected outright Trumps assertions that his legal issues amount to political persecution undertaken by the Biden administration What hes saying is not factual period she said And that would not be new for him would it And Harris said it was imperative Americans remain watchful for signs of democratic erosion in the leadup to this years contest"]

# Tokenize and predict for each example
for text in example_texts:
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: val.to(source_model.device) for key, val in inputs.items()}  # Send inputs to model's device

    # Source Prediction
    with torch.no_grad():
        source_outputs = source_model(**inputs)
        source_probabilities = torch.softmax(source_outputs.logits, dim=-1)  # Compute probabilities for each class

    # Get the most probable source label
    source_predictions = torch.argmax(source_probabilities, dim=-1).item()  # Get the index of the highest probability
    predicted_source_label = label_encoder_source.inverse_transform([source_predictions])[0]  # Decode the label

    # Get the top 3 most probable source labels and their probabilities
    top3_source_indices = torch.topk(source_probabilities, 3).indices[0].tolist()  # Indices of top 3 classes
    top3_source_probs = torch.topk(source_probabilities, 3).values[0].tolist()  # Probabilities of top 3 classes
    top3_source_labels = label_encoder_source.inverse_transform(top3_source_indices)  # Decode top 3 labels

    # Print the results
    print(f"Text: {text[:100]}...")  # Print first 100 characters of the text for brevity
    print(f"Predicted Source: {predicted_source_label}")
    print("Top 3 Source Predictions:")
    for label, prob in zip(top3_source_labels, top3_source_probs):
        print(f"- {label}: {prob:.4f}")
    print("-" * 20)


Text: Harris pins the blame on Trump for Supreme Court overturning Roe v Wade Vice President Kamala Harris...
Predicted Source: ABC News
Top 3 Source Predictions:
- ABC News: 0.5162
- ACLU: 0.3359
- AARP: 0.1478
--------------------
